In [8]:
import numpy as np
import cv2
from PIL import Image
import os
from inference import get_model
from inference_sdk import InferenceHTTPClient
from inference.models.utils import get_roboflow_model
from ultralytics import YOLO, settings

Setup Ultralytics. Check and set correct directory path.

In [9]:
print(settings)

value = settings["runs_dir"]
settings.update({"datasets_dir": "C:/Users/needh/OneDrive/Documents/Uni/2024/Sem 2 2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8"})

{'settings_version': '0.0.5', 'datasets_dir': 'C:/Users/needh/OneDrive/Documents/Uni/2024/Sem 2 2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8', 'weights_dir': 'weights', 'runs_dir': 'runs', 'uuid': '738b640563e1796fc30b0655092bc4a3040d093a0d04e9478cda7fb1552f2c1e', 'sync': True, 'api_key': '', 'openai_api_key': '', 'clearml': True, 'comet': True, 'dvc': True, 'hub': True, 'mlflow': True, 'neptune': True, 'raytune': True, 'tensorboard': True, 'wandb': True, 'vscode_msg': True}


Import Roboflow pre-trained dataset "Underwater Marine Species Computer Vision Project"

In [16]:
# Add your own image path to any arbitrary image. It currently only loads one image so we need to figure out how to load all images.
image_path = "C:/Users/needh/OneDrive/Documents/Uni/2024/Sem 2 2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8/train/images/288856187_d1fb57a113_b_jpg.rf.1a861f6bf38998d756e6f76a84b73340.jpg"

# Roboflow model
model_name = "underwater-marine-species"
model_version = "6"

# Get Roboflow face model (this will fetch the model from Roboflow)
model = get_roboflow_model(
    model_id="{}/{}".format(model_name, model_version),

    #Replace ROBOFLOW_API_KEY with your Roboflow API Key
    api_key="J7ljP04frhJRcBuovOmD"
)

In [17]:
frame = cv2.imread(image_path)

results = model.infer(image=frame,
                        confidence=0.5,
                        iou_threshold=0.5)

In [18]:
if results[0].predictions:
    prediction = results[0].predictions[0]
    print(prediction)
    
    x_center = int(prediction.x)
    y_center = int(prediction.y)
    width = int(prediction.width)
    height = int(prediction.height)

    # Calculate top-left and bottom-right corners from center, width, and height
    x0 = x_center - width // 2
    y0 = y_center - height // 2
    x1 = x_center + width // 2
    y1 = y_center + height // 2
    
    # Currently only takes one input text as a label
    cv2.rectangle(frame, (x0, y0), (x1, y1), (255,255,0), 10)
    cv2.putText(frame, "fish", (x0, y0 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)

x=335.0 y=330.5 width=388.0 height=387.0 confidence=0.9382820725440979 class_name='fish' class_confidence=None class_id=1 tracker_id=None detection_id='17724e4d-35e4-4569-8739-e5da2bd75d19' parent_id=None


Display chosen image.

In [19]:
# Show the image from image path. When we get multiple/all images to be loaded we don't want them all to be rendered simultaneously, so this will need to be modified.
cv2.imshow('Image Frame', frame)
cv2.waitKey(0) # waits until a key is pressed
cv2.destroyAllWindows() # destroys the window showing image